# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
This is a (binary) classification problem.

_Why?_

The codomain of the function being modeled is the finite set `{IS_OK, NEEDS_HELP}` ≈ `{0, 1}`.  

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [48]:
from __future__ import division

# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [49]:
n_students = len(student_data)

n_features = len(student_data.loc[0][:-1])

n_passed = len(student_data[student_data['passed'] == 'yes'])

n_failed = len(student_data[student_data['passed'] == 'no'])

assert n_passed == n_students - n_failed

grad_rate = 100.0 * n_passed / n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [50]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [51]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [52]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=95, random_state=42)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

Logistic Regression
------------
Logistic regression is a commonly used and interpretable classification algorithm. I have a strong interest in deep learning applications, which use layers of "artificial neurons" to learn features from data. For deep learning models used in classification tasks, the final layer is typically a logistic regressor (or, in the case of multinomial distributions, a "softmax" regressor). See the architecture for the GoogLeNet for an explicit example. http://www.cs.unc.edu/~wliu/papers/GoogLeNet.pdf

One advantage of logistic regression is that it may handle non-linear effects; plus, its interpretability is not much more complex than linear models. The model also makes very little assumptions about the underlying distribution. One disadvantage is that the MLE optimization routine can have trouble converging for certain problem instances, but these can often be addressed by tuning the learning rate or normalizing / rescaling the input data. Finally, the model relies on the discriminative power of existing features: this explains its utility in deep learning applications, where we use backprop and stochastic gradient descent to optimize the features learned by the model to acheive the best performance of the classifier.

I think this is a good fit for the problem for two main reasons. First, it is easy to interpret (later we will look at the coefficients derived from the learned model, and how they measure the relative the importance of the various predictor variables). Second, while we do not have a lot of data, I think we have enough to learn a good model, though probably not enough for more sophisticated techniques.

Support Vector Machine
------------
Support vector machines seek a separating hyperplane between the positive and negative data points that maximizes the "margin": the goal is to maximize the minimum distance between any data point and the separating hyperplane. I think this technique could work well for this problem for a few reasons: (1) the model is inheretly a binary classifier; (2) the hyperplane is defined only in terms of the "closest" points, and so it should be (somewhat) robust to the skew in our distribution (there are more students who passed than failed). Next, (3) despite the hyperplane ultimately being a linear separator, the plane will in fact exist in "kernel space": an abstract space defined by a metric function of our choosing. 

Unfortunately, this can lead to overfitting and an opaque interpretation. 

One real world use case of support vector machines is in high energy particle physics, in the problem of classifying "interesting" events (see "Support Vector Machines in High Energy Physics" https://arxiv.org/abs/0803.2345).


K-Nearest Neighbors (KNN)
------------
KNN works by making a prediction for each student based on the predictions of the "most similar" data points from training. This technique is limited in its scalability, since the memory requirements for the algorithm grow linearly with the training data. Given the size of our training data, this is not at present an issue. The assumption of the model is that points close together in feature space ought to have similar outcomes, which is not a bad assumption given the problem. I am including this technique as something of a baseline / sanity-check. Since the training time is virtually zero, it is very inexpensive to compare to our more scalable methods above. The low computational cost of prediciton and simplicity of the model are its strengths, and its lack of scalability is its main weakness. Another strength is that the learned function can incorporate non-linear aspects of the underlying distribution. Another weakness is that, over time, "stale" data points may be less relevant in predicting the outcome of nearby points, so continued use may require additional maintenance.

After some digging, I found the following interesting use case of KNN: detecting transister failures  https://www.researchgate.net/publication/3284325_Fault_Detection_Using_the_k-Nearest_Neighbor_Rule_for_Semiconductor_Manufacturing_Processes . Another more familiar example is KNN being used by Spotify for music recommendations. https://github.com/spotify/annoy

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [53]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    return end - start

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    return (end - start, f1_score(target.values, y_pred, pos_label='yes'))


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''    
    # Train the classifier
    train_time = train_classifier(clf, X_train, y_train)
    (train_predict_time, train_score) = predict_labels(clf, X_train, y_train)
    (test_predict_time, test_score) = predict_labels(clf, X_test, y_test)
    print " | {}   |    {:.4f}    |     {:.4f}      |   {:.4f}   |  {:.4f}    |".format(len(y_train),
                      train_time, train_predict_time, train_score, test_score )

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [54]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from skearln import model_C
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

clfs = (LogisticRegression(random_state=42), SVC(random_state=42), KNeighborsClassifier())
(lr, svc, knn) = clfs

# Execute the 'train_predict' function for each classifier and each training set size
i=0
for clf in clfs:        
    # Indicate the classifier and the training set size
    i += 1
    print "Classifier {} - ".format(i)
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))

    for j in range(1,4):
        train_predict(clf, X_train[:100*j], y_train[:100*j], X_test, y_test)
    print

Classifier 1 - 
Training a LogisticRegression using a training set size of 300. . .
 | 100   |    0.0023    |     0.0005      |   0.8593   |  0.7647    |
 | 200   |    0.0084    |     0.0020      |   0.8562   |  0.7914    |
 | 300   |    0.0049    |     0.0005      |   0.8468   |  0.8060    |

Classifier 4 - 
Training a SVC using a training set size of 300. . .
 | 100   |    0.0024    |     0.0017      |   0.8777   |  0.7746    |
 | 200   |    0.0045    |     0.0034      |   0.8679   |  0.7815    |
 | 300   |    0.0068    |     0.0076      |   0.8761   |  0.7838    |

Classifier 4 - 
Training a KNeighborsClassifier using a training set size of 300. . .
 | 100   |    0.0005    |     0.0012      |   0.8060   |  0.7246    |
 | 200   |    0.0005    |     0.0028      |   0.8800   |  0.7692    |
 | 300   |    0.0007    |     0.0060      |   0.8809   |  0.7801    |



### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.


 ** Classifer 1 - Logistic Regression**
 
 | Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
 | :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
 | 100   |    0.0023    |     0.0005      |   0.8593   |  0.7647    |
 | 200   |    0.0084    |     0.0020      |   0.8562   |  0.7914    |
 | 300   |    0.0049    |     0.0005      |   0.8468   |  0.8060    |
  
 ** Classifer 2 - Support Vector Machine**
 
 | Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
 | :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
 | 100   |    0.0024    |     0.0017      |   0.8777   |  0.7746    |
 | 200   |    0.0045    |     0.0034      |   0.8679   |  0.7815    |
 | 300   |    0.0068    |     0.0076      |   0.8761   |  0.7838    |
 
 ** Classifer 3 - K-Nearest Neighbors**
 
 | Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
 | :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
 | 100   |    0.0005    |     0.0012      |   0.8060   |  0.7246    |
 | 200   |    0.0005    |     0.0028      |   0.8800   |  0.7692    |
 | 300   |    0.0007    |     0.0060      |   0.8809   |  0.7801    |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

I will be optimizing a logistic regression classifier for this problem. Given the limited number of data points and relatively high dimensionality of the data, we would should desire interpretability and low model complexity.

Interpretability is a very desirable feature for a model for this specific problem. Presumably, we want not only to classify at risk students, but also "apply a treatment" to maximize their chances of passing. Being able to identify which features of the student are contributing to our belief in their risk of failing is important for "diagnosing" the at risk student.

The logistic regression model also shows consistent F1 score improvement as the number of data points increases, giving hope of further improvement at the future expense of more data collection. The SVM shows strong performance, and there is good reason to believe it may perform the best overall. However, the interpretability of the SVM is not as transparent as that of the LR classifier, and for this reason I am opting against using it. 

In reality, I would train and optimize both models, and possibly use them together in an ensemble.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

We train a model that learns a set of parameters that correspond to how certain evidence in an examples features correspond to the outcome being predicted. We learn one parameter for each feature. The higher the feature's parameter value, the more strongly it contributes to our belief in a student's risk/safety. The model is trained by iteratively changing the weights of the model to minimize error, until any change in the parameters leads to a negligible change in error (how we define "negligible" depends on the tolerance paramter; we will consider three different values in our optimization routine).

The exact relationship between the parameters and the weights requires a bit of mathematical jargon. In essence, each parameter indicates how much more likely we think the student is to pass/fail given a fixed increase/decrease in that parameter's value.

Explicitly: each learned parameter is the expected change in the log odds of passing for a unit increase in the corresponding feature, holding the other predictor variables constant at certain value. 

To make a prediciton, the model is given a new, unseen data point, performs a single linear algebra operation, and thresholds the output to determine our prediction of "pass" vs. "fail". 

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [56]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
# Running grid search on all classifiers, just out of curiosity

f1_scorer = make_scorer(f1_score, pos_label="yes")

parameter_dict = {svc.__class__.__name__:[
                         {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                          'C': [1, 10, 100, 1000]},
                         {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}], 
                  lr.__class__.__name__:
                         {'penalty':['l1','l2'], 
                          'tol':[0.001, 0.0001, 0.00001], 
                          'C':[0.1,0.6,1.0,2.0], 
                          'class_weight':[None,'balanced'], 
                          'n_jobs':[-1]}, 
                  knn.__class__.__name__:
                        {'n_neighbors':range(3,40,3),
                         'weights':['distance','uniform']}
                 }
for clf in clfs:
    start = time()
    print clf.__class__.__name__
    grid_obj = GridSearchCV(clf, parameter_dict[clf.__class__.__name__], scoring=f1_scorer)
    grid_obj.fit(X_train, y_train)
    # Get the estimator
    clf = grid_obj.best_estimator_
    end = time()
    # Report the final F1 score for training and testing after parameter tuning
    print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train)[1])
    print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test)[1])
    print "Grid search took " + str(end - start) + " seconds"

LogisticRegression
Tuned model has a training F1 score of 0.8323.
Tuned model has a testing F1 score of 0.7891.
Grid search took 1.35158896446 seconds
SVC
Tuned model has a training F1 score of 0.8323.
Tuned model has a testing F1 score of 0.7945.
Grid search took 108.40683794 seconds
KNeighborsClassifier
Tuned model has a training F1 score of 1.0000.
Tuned model has a testing F1 score of 0.7867.
Grid search took 0.455950975418 seconds


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The final score for the Logistic Regression model was 0.8323 for training and 0.7891 for testing. The final score is actually slightly worse than the untuned model. 

Curious, I ran the optimization on SVC and KNN to see how all three changed and which ultimiately did the best. All reached a similar score of around 0.8. I still think LR was a good choice, but in retrospect I do find the SVC attractive: the training complexity is higher than for LR but with the small data that is not too large a burden. The SVC performed the best, but I don't think the difference is very significant.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.